In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import peak_widths
from scipy.interpolate import interp1d
%matplotlib inline

In [ ]:
def read_raman_data(file_path):
    """
    Reads Raman data from a text file.

    Inputs:
    - file_path: Path to the text file containing Raman data.

    Returns:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing respective intensity values.
    - peak_threshold: Value calculated by average intensity added by some integer
    """
    data = np.loadtxt(file_path)
    raman_shift = data[:, 0]
    intensity = data[:, 1] # for map txt files, change the value to match which plot you'd like to visualize
    peak_threshold = np.average(intensity) + 10
    return raman_shift, intensity, peak_threshold

In [ ]:
def index_to_xdata(xdata, indices):
    "interpolate the values from signal.peak_widths to proper raman_shift data"
    ind = np.arange(len(xdata))
    f = interp1d(ind,xdata)
    return f(indices)

In [ ]:
def plot_raman_spectrum(raman_shift, intensity, peaks=None):
    """
    Plots the Raman spectrum.

    Inputs:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing Respective intensity values.
    - peaks: Dictionary containing peaks data, if it exists.
    """
    plt.figure(figsize=(11, 6))
    plt.plot(raman_shift, intensity, color='red', label='Raman Spectrum')

    if peaks:
        peak_indices = peaks['peak_indices']
        peak_values = peaks['peak_intensity']
        top_two_peaks = peaks['top_two_peaks']
        peak_ratio = peaks['peak_ratio']
        widths = peaks['peak_widths']
        width_heights = peaks['width_heights']
        left_ips = peaks['left_ips']
        right_ips = peaks['right_ips']
        peak_raman_shifts = peaks['peak_raman_shifts']
        plt.scatter(raman_shift[peak_indices], intensity[peak_indices], color='blue', marker='o', label='Peaks')
        for i, (shift, value) in enumerate(zip(raman_shift[peak_indices], peak_values)):
            plt.text(shift + 20, value, f'Peak {i+1}', ha='center', va='bottom', color='blue')
        for j in peak_raman_shifts:
            plt.axvline(x=j, color='black', linestyle='--', linewidth=1, label='Peak at {}'.format(j))
        plt.hlines(width_heights, left_ips, right_ips, color='g')

    plt.xlabel('Raman Shift (cm^-1)')
    plt.ylabel('Intensity (a.u.)')
    plt.title('Material') #For whatever material you're working with
    plt.annotate(f"Peak1/Peak2 ratio is {peak_ratio}", xy=(300, np.mean(peak_values)), xytext=(200, np.mean(peak_values)))
    plt.grid(True)
    plt.show()

In [ ]:
def identify_peaks(raman_shift, intensity, threshold, distance):
    """
    Identifies peaks in the Raman spectrum using scipy's find_peaks function.
    Identifies peak widths in the Raman spectrum using scipy's peak_widths function. 

    Inputs:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing corresponding intensity values.
    - threshold: Minimum intensity required to be considered as a peak.
    - distance: Minimum distance between peaks (in terms of array indices).

    Returns:
    - peaks: Dictionary containing peak indices, values, peak ratio, top two peaks, raman shift peak locations, width values per peak, width heights,
             and raman shift locations for widths
    """
    peaks, _ = find_peaks(intensity, height=threshold, distance=distance) 
    widths, width_heights, left_ips, right_ips = peak_widths(intensity, peaks, rel_height = 0.5) #adjust rel_height to find peak width at specific y vals 
    left_ips = index_to_xdata(raman_shift, left_ips)
    right_ips = index_to_xdata(raman_shift, right_ips)
    peak_intensity = intensity[peaks]
    peak_raman_shifts = raman_shift[peaks]
    sorted_peaks = sorted(peak_intensity, reverse =True)
    top_two_peaks = sorted_peaks[:2]
    peak_ratio = top_two_peaks[1]/top_two_peaks[0]
    return {'peak_indices': peaks, 'peak_intensity': peak_intensity, 'peak_ratio': peak_ratio, 'top_two_peaks': top_two_peaks, 
            'peak_raman_shifts': peak_raman_shifts, 'peak_widths': widths, 'width_heights': width_heights, 'left_ips': left_ips, 'right_ips': right_ips}

In [ ]:
if __name__ == "__main__":
    file_path = "path/to/file"  # Change this to your Raman data file path
    raman_shift, intensity, peak_threshold = read_raman_data(file_path)
    peaks = identify_peaks(raman_shift, intensity, threshold=peak_threshold, distance=55)  # Adjust threshold and distance as needed
    plot_raman_spectrum(raman_shift, intensity, peaks)

In [ ]:
peaks['peak_raman_shifts'] # these values actually tell you what the raman shift value is for each respective peak

In [ ]:
peaks['peak_indices'] # these values are extremely important because they highlight where the peaks are in our dataset

In [ ]:
peaks['peak_intensity'] # these values tell us where on the intensity axis the peaks are located

In [ ]:
peaks['peak_ratio'] # this value tells us the peak ratio from Peak1/Peak2

In [ ]:
peaks['peak_widths'] # this list tells us the width values for each peak in the plot, left to right